# Enable Model Monitoring
To see tracking results, Model Monitoring needs to be enabled in each model.

To enable Model Monitoring, include `serving_fn.set_tracking()` in the Model Server.

To utilize Drift measurement, supply the train set in the training step.

## Model Monitoring Demo
Use the following code blocks to test and explore Model Monitoring.

In [ ]:
# Set project name
project = "Demo-Project"

### Deploy Model Servers
Use the following code to deploy a model server in the Iguazio instance.

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

from mlrun import import_function, get_dataitem
from mlrun import projects
from mlrun.platforms import auto_mount

proj = projects.new_project(project)

# Download the pre-trained Iris model
get_dataitem("https://s3.wasabisys.com/iguazio/models/iris/model.pkl").download(
    "model.pkl")

iris = load_iris()
train_set = pd.DataFrame(iris['data'],
                         columns=['sepal_length_cm', 'sepal_width_cm',
                                  'petal_length_cm', 'petal_width_cm'])

# Import the serving function from the function hub
serving_fn = import_function('hub://v2_model_server').apply(auto_mount())

name = "RandomForestClassifier"

# Log the model through the projects API so that it is available through the feature store API
proj.log_model(name, model_file="model.pkl", training_set=train_set)

# Add the model to the serving function's routing spec
serving_fn.add_model(name, model_path=f"store://models/{project}/{name}:latest")

# Change the default project name of the serving function
serving_fn.metadata.project = project

# Enable model monitoring
serving_fn.set_tracking()

# Deploy the function
serving_fn.deploy()

### Deploy Stream Processing
Use the following code to create statistical data from the model endpoints to be presented in the dashboards.

In [ ]:
import os

from mlrun import import_function
from mlrun.platforms import mount_v3io
from mlrun.runtimes import RemoteRuntime
import json

fn: RemoteRuntime = import_function("hub://model_monitoring_stream")

# Configures the nuclio trigger. See nuclio triggers https://nuclio.io/docs/latest/reference/triggers/.
fn.add_v3io_stream_trigger(
    stream_path=f"projects/{project}/model-endpoints/stream",
    name="monitoring_stream_trigger",
)

fn.set_env("MODEL_MONITORING_PARAMETERS", json.dumps(
    {"project": project, "v3io_framesd": os.environ.get("V3IO_FRAMESD")}))

fn.metadata.project = project
fn.apply(mount_v3io())

# Deploy the function
fn.deploy()

### Deploy Batch Processing
Use the following code calculate concept drift based on the statistical data from the model endpoints.

In [ ]:
from mlrun import import_function
from mlrun.platforms import mount_v3io
from mlrun.runtimes import KubejobRuntime

fn: KubejobRuntime = import_function("hub://model_monitoring_batch")
fn.metadata.project = project
fn.apply(mount_v3io())

# Run the function as an MLRun job
fn.run(name='model-monitoring-batch', schedule="0 */1 * * *",
       params={"project": project})

### Simulating Requests
Use the following code to simulate production data.

In [ ]:
import json
from time import sleep
from random import choice, uniform
from sklearn.datasets import load_iris

iris = load_iris()
iris_data = iris['data'].tolist()

while True:
    data_point = choice(iris_data)
    serving_fn.invoke(f'v2/models/{name}/infer', json.dumps({'inputs': [data_point]}))
    sleep(uniform(0.2, 1.7))
